# SQL HW

## Task 1 — Familiarize with SQL Clauses

### Sort data by start_station_id, tripduration 
SELECT * FROM citibike

WHERE tripduration <= 10800

ORDER BY start_station_id

### Only show the top/last 10 records (aka head and tail in SQL) 

**Head:**

SELECT * FROM citibike

WHERE tripduration <= 10800

ORDER BY start_station_id

LIMIT 10 

**Tail:**

SELECT * FROM citibike

WHERE tripduration <= 10800

ORDER BY start_station_id DESC

LIMIT 10 

### List all unique start_station_id values

SELECT DISTINCT start_station_id FROM citibike

### Count the number of trips (aka wc -l in SQL)
SELECT COUNT(tripduration) FROM citibike

### Find the average/min/max trip duration
SELECT AVG(tripduration) as myavg, MAX(tripduration) as mymax, MIN(tripduration) as mymin FROM citibike

## Task 2 — Working with date/time

### Selecting trips started on Feb-02-2015 only

SELECT * FROM citibike

WHERE starttime >= '2015-02-02 00:00'

AND starttime < '2015-02-03 00:00'

### Selecting trips started on the weekends 

SELECT * FROM citibike

WHERE extract(DOW FROM starttime) in (0,6)

### Average Trip Duration on Weekends

SELECT AVG(tripduration) as weekend_avg FROM citibike

WHERE extract(DOW FROM starttime) in (0,6) 

### Average Trip Duration on Weekdays

SELECT AVG(tripduration) as weekend_avg FROM citibike

WHERE extract(DOW FROM starttime) not in (0,6) 

## Task 3 — Working with Space

### Showing the list of start station locations 

SELECT COUNT(cartodb_id)

GROUP BY start_station_latitude, start_station_longitude

### Showing the number of trips started per station

SELECT COUNT(tripduration) FROM citibike

GROUP BY start_station_latitude, start_station_longitude

### Showing the number of trips started per station 500 M from Times Square

SELECT CDB_TransformToWebmercator(CDB_LatLng(start_station_latitude, start_station_longitude)) as the_geom_webmercator, 
	MIN(cartodb_id) as cartodb_id,
    COUNT(tripduration) as numtrips

FROM citibike

WHERE ST_DWithin(
  		CDB_LatLng(start_station_latitude, start_station_longitude)::geography,
  		CDB_LatLng(40.7577,-73.9857)::geography,
  		500)
        
GROUP BY start_station_latitude, start_station_longitude

## Task 4 — Putting it all together
### Find the station that had the longest average trip duration during the weekend and within 500m of TimeSquare! 

SELECT CDB_TransformToWebmercator(CDB_LatLng(start_station_latitude, start_station_longitude)) as the_geom_webmercator, 
	MIN(start_station_id) as station_id,
    COUNT(tripduration) as numtrips,
    AVG(tripduration) as avg_tripduration

FROM citibike

WHERE ST_DWithin(
  		CDB_LatLng(start_station_latitude, start_station_longitude)::geography,
  		CDB_LatLng(40.7577,-73.9857)::geography,
  		500)
        AND extract(DOW FROM starttime) in (0,6)
        

GROUP BY start_station_latitude, start_station_longitude

ORDER by avg_tripduration desc

LIMIT 1

### create lines for trips started from stations within 500m of Times Squares and lasted less than 2 hours.

SELECT ST_MakeLine(CDB_TransformToWebmercator(CDB_LatLng(start_station_latitude, start_station_longitude)),
                   CDB_TransformToWebmercator(CDB_LatLng(end_station_latitude, end_station_longitude)))
    as the_geom_webmercator,
	MIN(cartodb_id) as cartodb_id,
    COUNT(tripduration) as numtrips

FROM citibike

WHERE ST_DWithin(
  		CDB_LatLng(start_station_latitude, start_station_longitude)::geography,
  		CDB_LatLng(40.7577,-73.9857)::geography,
  		500)
        
        AND tripduration<7200
        
GROUP BY start_station_latitude, start_station_longitude, end_station_latitude, end_station_longitude
